In [1]:
import os
import pandas as pd
import subprocess

def create_parser():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "bedfile",
        type=str,
     )

    parser.add_argument(
        "genelist",
        type=str,
    )

    parser.add_argument(
        "genomefasta",,
    )
    
    parser.add_argument(
         "outputfasta",
         type = str,
    )
    parser.add_argument(
         "workingdir",
         type = str,
    )
    return parser

In [3]:
with open('CH.bed', 'r') as fin:
    data = fin.read().splitlines(True)
with open('CH2.bed', 'w') as fout:
    fout.writelines(data[1:])

In [4]:
df = pd.read_csv('CH2.bed',sep='\t', names = ['chrom', 'chromStart' ,'chromEnd' ,'name' ,'score' ,'strand' ,'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts'])

In [5]:
df = df.loc[:, 'chrom':'strand']

In [6]:
with open('/media/user/A057-7A86/untitled folder/genelist_query_x.txt','r') as fn:
    GeneList = fn.read()
    GeneList = GeneList.split('\n')

In [7]:
data = df['name'].isin(GeneList)
entr = []
for idx, i in enumerate(data.values):
    if i == True:
        entr.append(list(data.index)[idx])

In [8]:
df2 = df.iloc[entr,:]

In [10]:
df2[df2['strand']=='-']

,chrom,chromStart,chromEnd,name,score,strand
662085,CM004455.1_2,5135547.0,5135651.0,OBR16331.1,0.0,-
662086,CM004455.1_2,5135259.0,5135459.0,OBR16331.1,0.0,-
662087,CM004455.1_2,5135011.0,5135170.0,OBR16331.1,0.0,-
662088,CM004455.1_2,5134359.0,5134904.0,OBR16331.1,0.0,-
662089,CM004455.1_2,5133978.0,5134252.0,OBR16331.1,0.0,-
...,...,...,...,...,...,...
757996,LTAN01000007.1_2,2890095.0,2890386.0,OBR06459.1,0.0,-
757997,LTAN01000007.1_2,2889945.0,2890029.0,OBR06459.1,0.0,-
757998,LTAN01000007.1_2,2889775.0,2889885.0,OBR06459.1,0.0,-
757999,LTAN01000007.1_2,2888641.0,2889717.0,OBR06459.1,0.0,-


In [12]:
sel_gen = list(df2['name'])
ingenes = list(set([i for i in sel_gen if sel_gen.count(i) > 1])) #list of genes with introns
for idx,_ in enumerate(ingenes):
    sl = df2[df2['name']==ingenes[idx]]
    sl.reset_index(inplace = True)
    df2.drop(labels= df2.index[df2['name']==ingenes[idx]].tolist(), axis=0, inplace=True)
    sub=pd.Series([sl.loc[len(sl)-1]['chrom'],sl.loc[len(sl)-1]['chromStart'], sl.loc[len(sl)-1]['chromEnd'],ingenes[idx],sl.loc[len(sl)-1]['score'],sl.loc[len(sl)-1]['strand']],index = df2.columns)
    df2=df2.append(sub,ignore_index=True)
    del sl

In [13]:
df2

,chrom,chromStart,chromEnd,name,score,strand
0,CM004455.1_2,5968882.0,5969635.0,OBR16580.1,0.0,+
1,CM004455.1_2,3354911.0,3356948.0,OBR15833.1,0.0,-
2,CM004456.1_2,168034.0,168772.0,OBR13103.1,0.0,-
3,CM004457.1_2,4418494.0,4420885.0,OBR12554.1,0.0,+
4,CM004457.1_2,5707409.0,5709836.0,OBR12972.1,0.0,-
...,...,...,...,...,...,...
152,CM004456.1_2,5712573.0,5712671.0,OBR14729.1,0.0,-
153,LTAN01000007.1_2,3878844.0,3879820.0,OBR06742.1,0.0,+
154,CM004459.1_2,4661762.0,4662515.0,OBR09495.1,0.0,+
155,CM004455.1_2,596054.0,597417.0,OBR15006.1,0.0,+


In [14]:
pd.options.mode.chained_assignment = None
for idx in df2['chromStart'].index:
    chromStart = df2['chromStart'][idx]
    chromEnde = df2['chromEnd'][idx]
    if chromStart >1000 and df2['strand'][idx] == '+':
        df2['chromStart'][idx] = chromStart-1000
        df2['chromEnd'][idx] = chromStart
    if df2['strand'][idx] == '-':
        df2['chromStart'][idx] = chromEnde
        df2['chromEnd'][idx] = chromEnde+1000

In [15]:
df2['chromStart'] = df2['chromStart'].astype('int')
df2['chromEnd'] = df2['chromEnd'].astype('int')

In [16]:
df2.to_csv('/home/user/Desktop/test.bed', sep ='\t' , index=False, header=False)

In [19]:
p = subprocess.Popen(["sh","./bedtools.sh"," /home/user/Desktop/CH.fasta", "/home/user/Desktop/test.bed",
                      "/home/user/Desktop/Promotor_1.fasta"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

In [20]:
stdout, stderr = p.communicate()
print("stdout: '%s'" % stdout)
print("stderr: '%s'" % stderr)

stdout: 'b'Finished Promotor Sequence Extraction\n''
stderr: 'None'
